# Building a Quasi-Agent

A quasi-agent is like a lightweight version of an AI agent. It acts as a smart assistant that can follow instructions step by step,

but it can’t make decisions or learn from feedback like a full-fledged agent would.

In this case, we’re building one that:

- Asks the user to provide a customer complaint (like a chatbot)
- Generates a professional and empathetic initial response
- Enhances the tone and adds personalization to the message
- Appends a compensation or resolution plan to retain customer trust

It’s a controlled simulation of an AI assistant with memory, 

built to practice how to maintain context across steps and manage multi-turn interactions using prompt chaining.

------------------------------------------------------------------------------------------------------------

# Quasi-Agent Architecture:

The goal is to simulate a customer service agent that helps respond to unhappy customers by walking through 3 distinct steps:

1- Initial Complaint Response Generation: {The quasi-agent takes a customer complaint as input and generates the response.}

2- Tone and Personalization Enhancement: { 1. The draft from Step 1 is passed as input again.

                                      2. The agent improves the tone friendly and adds personal touches, such as using the customer’s name or order details, to make the response more human and caring.}

3- Add Compensation or Action Plan: { 1. The enhanced message is passed to the agent once more.

                                  2. The agent appends an apology gift to help retain customer trust and satisfaction.}

In [8]:
import os 
from litellm import completion
from typing import List, Dict

In [9]:
import os
from dotenv import load_dotenv

# Get API key from environment variables
load_dotenv()
api_key = os.environ.get('OPENAI_API_KEY')

if api_key:
    print("API key loaded successfully!")
else:
    print("Error: API key not found in .env file")


API key loaded successfully!
